In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy import signal
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
import numpy as np


In [2]:
path = "C:/Users/w/OneDrive/Documents/Second Term/CDSS/Final Project ADHD/DREAMER.mat"
raw = sio.loadmat(path)

In [3]:
def preprocess_EEG(raw, feature):
    overall = signal.firwin(9, [0.0625, 0.46875], window="hamming")
    theta = signal.firwin(9, [4, 8], fs=250, window="hamming")
    alpha = signal.firwin(9, [8, 13], fs=250, window="hamming")
    beta = signal.firwin(9, [13, 30], fs=250, window="hamming")
    filt_data = signal.filtfilt(overall, 1, raw)
    filt_theta = signal.filtfilt(theta, 1, filt_data)
    filt_alpha = signal.filtfilt(alpha, 1, filt_data)
    filt_beta = signal.filtfilt(beta, 1, filt_data)
    ftheta, psdtheta = signal.welch(filt_theta, fs =128,nperseg=256)
    falpha, psdalpha = signal.welch(filt_alpha, fs =128,nperseg=256)
    fbeta, psdbeta = signal.welch(filt_beta,fs =128, nperseg=256)
    feature.append(np.mean(psdtheta))
    feature.append(np.mean(psdalpha))
    feature.append(np.mean(psdbeta))
    return feature

In [4]:
def feat_extract_EEG(raw):
    EEG_tmp = np.zeros((23, 18, 42))
    for participant in range(0, 23):
        for video in range(0, 18):
            for i in range(0, 14):
                B, S = [], []
                basl = (raw["DREAMER"][0, 0]["Data"]
                        [0, participant]["EEG"][0, 0]
                        ["baseline"][0, 0][video, 0][:, i])
                stim = (raw["DREAMER"][0, 0]["Data"]
                        [0, participant]["EEG"][0, 0]
                        ["stimuli"][0, 0][video, 0][:, i])
                B = preprocess_EEG(basl, B)
                S = preprocess_EEG(stim, S)
                Extrod = np.divide(S, B)
                EEG_tmp[participant, video, 3*i] = Extrod[0]
                EEG_tmp[participant, video, 3*i+1] = Extrod[1]
                EEG_tmp[participant, video, 3*i+2] = Extrod[2]
    col = []
    for i in range(0, 14):
        col.append("psdtheta_"+str(i + 1)+"_un")
        col.append("psdalpha_"+str(i + 1)+"_un")
        col.append("psdbeta_"+str(i + 1)+"_un")

    data_EEG = pd.DataFrame(EEG_tmp.reshape((23 * 18,
                                                EEG_tmp.shape[2])), columns=col)
    scaler = StandardScaler()
    for i in range(len(col)):
        data_EEG[col[i][:-3]] = scaler.fit_transform(data_EEG[[col[i]]])
    data_EEG.drop(col, axis=1, inplace=True)
    return data_EEG

In [5]:
def participant_affective(raw):
    a = np.zeros((23, 18, 9), dtype=object)
    for participant in range(0, 23):
        for video in range(0, 18):
            a[participant, video, 0] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Age"][0][0][0])
            
            a[participant, video, 1] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Gender"][0][0][0])
            
            a[participant, video, 2] = int(participant+1)
            
            a[participant, video, 3] = int(video+1)
            
            a[participant, video, 4] = ["Searching for Bobby Fischer",
                                        "D.O.A.", "The Hangover", "The Ring",
                                        "300", "National Lampoon\'s VanWilder",
                                        "Wall-E", "Crash", "My Girl",
                                        "The Fly", "Pride and Prejudice",
                                        "Modern Times", "Remember the Titans",
                                        "Gentlemans Agreement", "Psycho",
                                        "The Bourne Identitiy",
                                        "The Shawshank Redemption",
                                        "The Departed"][video]
            
            a[participant, video, 5] = ["calmness", "surprise", "amusement",
                                        "fear", "excitement", "disgust",
                                        "happiness", "anger", "sadness",
                                        "disgust", "calmness", "amusement",
                                        "happiness", "anger", "fear",
                                        "excitement", "sadness",
                                        "surprise"][video]
            a[participant, video, 6] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreValence"]
                                           [0, 0][video, 0])
            a[participant, video, 7] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreArousal"]
                                           [0, 0][video, 0])
            a[participant, video, 8] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreDominance"]
                                           [0, 0][video, 0])
    b = pd.DataFrame(a.reshape((23*18, a.shape[2])),
                     columns=["age", "gender", "participant",
                              "video", "video_name", "target_emotion",
                              "valence", "arousal", "dominance"])
    return b

In [6]:
df_features = feat_extract_EEG(raw)
df_participant_affective = participant_affective(raw)

df_participant_affective["valence"] = (df_participant_affective
                                       ["valence"].astype(int))
df_participant_affective["arousal"] = (df_participant_affective
                                       ["arousal"].astype(int))
df_participant_affective["dominance"] = (df_participant_affective
                                         ["dominance"].astype(int))
df = pd.concat([df_features, df_participant_affective], axis=1)

In [7]:
df

,psdtheta_1,psdalpha_1,psdbeta_1,psdtheta_2,psdalpha_2,psdbeta_2,psdtheta_3,psdalpha_3,psdbeta_3,psdtheta_4,...,psdbeta_14,age,gender,participant,video,video_name,target_emotion,valence,arousal,dominance
0,-0.150843,-0.150624,-0.166261,-0.145117,-0.143640,-0.119170,-0.475254,-0.468150,-0.330365,-0.427103,...,-0.204449,22,male,1,1,Searching for Bobby Fischer,calmness,4,3,2
1,-0.128521,-0.129717,-0.155580,-0.105668,-0.108871,-0.168438,0.302383,0.258150,-0.121029,-0.052994,...,-0.166154,22,male,1,2,D.O.A.,surprise,3,3,1
2,-0.103209,-0.103515,-0.122637,-0.109388,-0.111235,-0.140875,0.063157,0.057745,0.006707,-0.081610,...,-0.051510,22,male,1,3,The Hangover,amusement,5,4,4
3,-0.181723,-0.184244,-0.230825,-0.113030,-0.114522,-0.139100,-0.067104,-0.068218,-0.071644,-0.130311,...,-0.153138,22,male,1,4,The Ring,fear,4,3,2
4,-0.043894,-0.042331,-0.032673,-0.148676,-0.152181,-0.201394,0.314590,0.301284,0.191730,-0.023894,...,0.108396,22,male,1,5,300,excitement,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,-0.259796,-0.264659,-0.351959,-0.215325,-0.223008,-0.347720,-0.282586,-0.281040,-0.263415,-0.350000,...,-0.049186,25,male,23,14,Gentlemans Agreement,anger,2,2,2
410,-0.220586,-0.227271,-0.333407,-0.083146,-0.084745,-0.115249,-0.452527,-0.461702,-0.518178,-0.551788,...,0.012865,25,male,23,15,Psycho,fear,2,2,2
411,-0.054579,-0.056091,-0.082380,0.014625,0.014887,0.025082,0.225323,0.234515,0.292967,0.233246,...,-0.061149,25,male,23,16,The Bourne Identitiy,excitement,3,3,2
412,-0.242060,-0.244057,-0.288320,-0.134908,-0.139699,-0.208673,-0.238502,-0.241715,-0.257637,-0.356707,...,-0.121816,25,male,23,17,The Shawshank Redemption,sadness,2,2,4


In [9]:
data = df.loc[(df['target_emotion'] == 'anger') |
                (df['target_emotion'] == 'fear') |
                (df['target_emotion'] == 'calmness')|
                (df['target_emotion'] == 'surprise') |
                (df['target_emotion'] == 'excitement') |
                (df['target_emotion'] == 'amusement') |
                (df['target_emotion'] == 'happiness') |
                (df['target_emotion'] == 'sadness') |
                (df['target_emotion'] == 'disgust')].copy()

In [10]:
data['stress_bin'] = data['target_emotion'].map(
    {'surprise': 0, 'excitement': 0, 'amusement': 0, 'happiness': 0,'anger': 1, 'fear': 1, 'disgust':2, 'calmness': 0}
)

In [11]:
# save features, demographic, and emotion data as csv
data.to_csv('ml_data.csv')